# IAM Data Science Pre-Workshop Linear Algebra Lecture #3

### Switched to Python 3

## Usual Slate of Package Initialization

In [ ]:
import scipy as sp
import scipy.linalg as la
import numpy as np
import math
import matplotlib
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import time
from PIL import Image
%matplotlib inline
np.set_printoptions(precision=3)

## Motivation of Eigenanalysis from Markov Processes

$4 \times 4$ transition matrix

In [ ]:
P = np.array([[0,1/4,1/2,0], [1/2,1/4,0,1/2],[1/2,1/4,0,0],[0,1/4,1/2,1/2]])
print(P)

Start at state 1 and look at the state at time $\alpha$ and $P^\alpha$

In [ ]:
alpha = 2000
Palpha = np.linalg.matrix_power(P,alpha)
x0 = np.array([1.,0,0,0])
xalpha = np.dot(Palpha,x0)
print('state:', xalpha)
print('\nalpha power of P\n',Palpha)

Now do the eigenanalysis of P

In [ ]:
(v,r) = la.eig(P)
fact = np.sum(r[:,0])
r[:,0] = r[:,0]/fact # I scale the first eigenvector to make it a probability vector
print('Eigenvalues:', v)
print('\nEigenvectors in columns:\n',r)

Check that it did what we think it should (i.e. verify that `r[:,0]` really is an eigenvector).

In [ ]:
print(np.dot(P,r[:,0]))

In [ ]:
print('Original P:\n',P, "\n")
print('RDRinv =P?:\n',r@sp.diag(v)@np.linalg.inv(r))

That's hard to parse because of entries like `-1.203e-16+0.j`, so we can use `numpy`'s `round` function, and only look at the real entries (because we can see the complex ones vanish.

In [ ]:
print('Original P:\n',P, "\n")
print('RDRinv =P?:\n',(r@sp.diag(v)@np.linalg.inv(r)).real.round(2))

## Symmetric Matrices

In [ ]:
A = np.array([[1,2],[2,1]])
print('Matrix: \n',A)
(v,r) = np.linalg.eig(A)
print('\nEigenanalysis:', v, "\n")
print(r)

Example from the 1D finite difference discretization from lecture #1

In [ ]:
N = 8;
pi = math.pi
h = 2*pi/N     # grid spacing
fact = 1/(h**2)
ond = np.zeros(N)+2*fact+1;
offd = np.zeros(N-1)-fact;
A = np.diag(ond,0)+np.diag(offd,1)+np.diag(offd,-1) # matrix is mostly tridiagonal
A[0,N-1] = -fact # terms from periodicity
A[N-1,0] = -fact
(v,r) = la.eig(A)
print('Eigenanalysis:\n',v)
print('n',r)

# Transform Methods

Same 1D code from lecture 1

In [ ]:
N = 16;
pi = math.pi
h = 2*pi/N     # grid spacing
fact = 1/(h**2)
ond = np.zeros(N)+2*fact+1;
offd = np.zeros(N-1)-fact;
A = np.diag(ond,0)+np.diag(offd,1)+np.diag(offd,-1) # matrix is mostly tridiagonal
A[0,N-1] = -fact # terms from periodicity
A[N-1,0] = -fact
x = np.arange(N)*h
uexact = np.zeros(N) # exact solution
rhs = np.zeros(N) # RHS f that matches the exact solution
uexact = np.exp(np.cos(x))
rhs = uexact*(np.cos(x)+np.cos(x)**2) 
u = np.linalg.solve(A, rhs) # solve 
error = np.amax(np.absolute(u-uexact)) # maximum norm of the error
print('Maximum norm error %10.4e' % error)

Solve based on the DFT diagonalization

In [ ]:
alpha = np.arange(N)
symbol = (2-2*np.cos(alpha*h))/h**2 +1
rhs_hat = np.fft.fft(rhs)
u_hat = rhs_hat/symbol
u= np.fft.ifft(u_hat)
error = np.amax(np.absolute(u-uexact)) # maximum norm of the error
print('Maximum norm error %10.4e' % error)

Spectrally accurate approximation 

In [ ]:
alpha2 = np.concatenate((np.arange(N/2+1), np.arange(-N/2+1,0,1)))
symbol = alpha2**2 +1
rhs_hat = np.fft.fft(rhs)
u_hat = rhs_hat/symbol
u= np.fft.ifft(u_hat)
error = np.amax(np.absolute(u-uexact)) # maximum norm of the error
print('Maximum norm error %10.4e' % error)

## Singular Value Decomposition 

### Geometrical interpretations of the SVD

video: https://www.youtube.com/watch?v=NsNNI_-JPUY

algebraic: it is a rank one decomposition
$$
\sum_{i=1}^n \sigma_i u_i v_i^T 
$$

#### If A is SPD then SVD is just eigenanlysis

In [ ]:
A = np.array([[3,-1],[-1,3]])
(v,r) = np.linalg.eig(A)
print('Eigenanalysis:\n',v)
print('\n',r)
(u,s,v) = np.linalg.svd(A)
print('\n \n SVD u, s, v\n',u)
print('\n',s)
print('\n',v)

#### If A is symmetric but has eigenvalues of mixed sign

In [ ]:
A = np.array([[1,2],[2,1]])
(v,r) = np.linalg.eig(A)
print('Eigenanalysis:\n',v)
print('\n',r)
(u,s,v) = np.linalg.svd(A)
print('\n \n SVD u, s, v\n',u)
print('\n',s)
print('\n',v)

#### If A is not diagonalizable

In [ ]:
A = np.array([[1,1],[0,1]])
(v,r) = np.linalg.eig(A)
print('Eigenanalysis:\n',v)
print('\n',r)
(u,s,v) = np.linalg.svd(A)
print('\n \n SVD u, s, v\n',u)
print('\n',s)
print('\n',v)

## SVD for least squares

In [ ]:
N = 20
h = 1./N
noise = 0.1
x = np.arange(N)*h
yexact = np.exp(x)
y = yexact+noise*np.random.normal(size=x.shape)
xmatrix = np.column_stack([x, np.ones(N)])
(u,s,v) = np.linalg.svd(xmatrix)
smatrix = np.zeros((2,N))
smatrix[0,0] = 1/s[0]
smatrix[1,1] = 1/s[1]
z = v.T@smatrix@u.T@y
m = z[0]
b = z[1]
print ('from SVD: m: %10.6f b: %10.6f' % (m,b))
yfit = m*x+b
plt.figure(1)
plt.plot(x,yexact,x,y,'ro',x,yfit)
plt.show()
m, b = np.linalg.lstsq(xmatrix,y)[0]
print ('from least squares package m: %10.6f b: %10.6f' % (m,b))

## SVD for Image compression

Code courtesy of Frank Cleary, taken directly from 
http://www.frankcleary.com/svdimage/

Jpeg image of my old father (90 years old)

In [ ]:
img = Image.open('dad_intellectual.JPG')
imggray = img.convert('LA')
plt.figure(figsize=(9, 6))
plt.imshow(imggray);
print('Image size:',imggray.size[1],' by ', imggray.size[0])

Turn it into a floating point matrix, does not change the image

In [ ]:
imgmat = np.array(list(imggray.getdata(band=0)), float)
imgmat.shape = (imggray.size[1], imggray.size[0])
imgmat = np.matrix(imgmat)
plt.figure(figsize=(9,6))
plt.imshow(imgmat, cmap='gray');

Perform the SVD

In [ ]:
U, sigma, V = np.linalg.svd(imgmat)

Now reconstruct the image using only i terms of the rank 1 decomposition

In [ ]:
i=50
reconstimg = np.matrix(U[:, :i]) * np.diag(sigma[:i]) * np.matrix(V[:i, :])
plt.imshow(reconstimg, cmap='gray')
title = "n = %s" % i
plt.title(title)
plt.show()
